In [1]:
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from joblib import dump, load
import queue
import pandas as pd
import numpy as np
import glob, os, sys
pd.options.mode.chained_assignment = None


c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [21]:
with open('./Dataset/csv/Dos_attack.csv', newline='') as csvfile:
        rows = pd.read_csv(csvfile,header=None)
        y = rows[10]
        x = rows.drop([10], axis=1)
        X_train, X_test, y_train, y_test = train_test_split(x, y , test_size=0.2)
    
if not os.path.exists("./models"):
    os.mkdir("./models")

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/DoS_DT.joblib')
prediction = model.predict(X_test)
print(classification_report(y_test,prediction))

1.0
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     22781
         1.0       1.00      1.00      1.00      6087

    accuracy                           1.00     28868
   macro avg       1.00      1.00      1.00     28868
weighted avg       1.00      1.00      1.00     28868



In [24]:
with open('./Dataset/csv/Spoofing_brakemax.csv', newline='') as csvfile:
        rows = pd.read_csv(csvfile,header=None)
        y = rows[10]
        x = rows.drop([10], axis=1)
        X_train, X_test, y_train, y_test = train_test_split(x, y , test_size=0.2)
    
if not os.path.exists("./models"):
    os.mkdir("./models")

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/Spoofing_DT.joblib')
prediction = model.predict(X_test)
print(classification_report(y_test,prediction))

0.99546210336705
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     22822
         1.0       0.99      0.98      0.99      6046

    accuracy                           1.00     28868
   macro avg       1.00      0.99      0.99     28868
weighted avg       1.00      1.00      1.00     28868



In [25]:
model = LogisticRegression(penalty="l2" ,C=202010.51396976886)
model.fit(X_train, y_train)
print(model.predict_proba(X_test)[:,1])
print(model.score(X_test, y_test))
dump(model, './models/DoS_LR.joblib')
prediction = model.predict(X_test)
print(classification_report(y_test,prediction))

[2.48152492e-09 1.09161394e-06 4.08459116e-09 ... 4.70750462e-08
 1.09160005e-06 8.44884251e-01]
0.9653249272550921
              precision    recall  f1-score   support

         0.0       1.00      0.96      0.98     22822
         1.0       0.86      1.00      0.92      6046

    accuracy                           0.97     28868
   macro avg       0.93      0.98      0.95     28868
weighted avg       0.97      0.97      0.97     28868



In [26]:
model = XGBClassifier(booster="gbtree", min_split_loss=7.100747859845302e-07, alpha=0.0010277375135306342, 
                        max_depth=9, eta=0.6737504946980999, gamma=6.101266632438708e-07, grow_policy="lossguide")
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
dump(model, './models/DoS_XGB.joblib')
prediction = model.predict(X_test)
print(classification_report(y_test,prediction))

c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[19:59:11] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9959124289871137
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     22822
         1.0       0.99      0.99      0.99      6046

    accuracy                           1.00     28868
   macro avg       1.00      0.99      0.99     28868
weighted avg       1.00      1.00      1.00     28868



c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
